In [5]:
import json
from ensemble_boxes import *
import argparse
import glob
import os
import pickle
import cv2

def getBoxes(data):
    box_with_image_id = {}
    for result in data:
        img_id = result["image_id"]
        detection = {"bbox": result["bbox"], "category_id": result["category_id"], "score":result["score"]}
        if not img_id in box_with_image_id.keys():
            box_with_image_id[img_id] = [detection]
        else:
            box_with_image_id[img_id].append(detection)
    return box_with_image_id

def xyxy2xywh(box):
    x,y,w,h = box
    return [int(x), int(y), int(x+w), int(y+h)]

def CocoBoxFromNormBox(normBox, width, height):
    x1, y1, x2, y2 = normBox
    return [int(x1*width+0.5), int(y1*height+0.5), int((x2-x1)*width+0.5), int((y2-y1)*height+0.5)]

def getNormBoxes(bboxes, image_id, width, height):
    all_boxes = bboxes[image_id]
    box = []
    class_id = []
    score = []
    for box_det in all_boxes:
#         if box_det['score'] == 0: continue
        x1,y1,x2,y2 = xyxy2xywh(box_det['bbox'])
        box.append([x1/width, y1/height, x2/width, y2/height])
        class_id.append(box_det['category_id'])
        score.append(box_det['score'])
    return box, class_id, score

def toJson(image_id, boxes, class_ids, scores, width, height):
    result = []
    for i in range(len(class_ids)):
        json_str = {}
        json_str["image_id"] = image_id
        json_str["category_id"] = int(class_ids[i])
        json_str["score"] = float(scores[i])
        json_str["bbox"] = CocoBoxFromNormBox(boxes[i], width, height)
        result.append(json_str)
    return result

def load_all_dets(all_pred_jsons):
    all_dets = []
    weights = []
    for det_json in all_pred_jsons:
#         if 'fold_1' in det_json:
#             weights.append(1)
#         elif 'fold_3' in det_json:
#             weights.append(3)
#         else:
#             weights.append(2)
        weights.append(1)

        with open(det_json, 'r') as f:
            prediction = json.load(f)
            all_dets.append(getBoxes(prediction))

    for i in range(len(weights)):
        print(all_pred_jsons[i], weights[i])
    return all_dets, weights






In [6]:
def wbf(all_dets, iou_thr=0.9, skip_box_thr=0.001, weights = None):
    data_p_dir = 'data.p'
    with open(data_p_dir, 'rb') as fp:
        data_ = pickle.load(fp)    
        
    data = []
    image_ids = set().union(*all_dets)
    for image_id in image_ids:
#         print(image_id)
#         width = data_['{}.dicom.png'.format(image_id)]['width']
#         height = data_['{}.dicom.png'.format(image_id)]['height']
        
        height, width, _ = cv2.imread('/home/hana/sonnh/kaggle-vin/dataset/images_only/test/{}.dicom.png'.format(image_id[:-6])).shape
    
        boxes_to_fusion = []
        class_to_fusion = []
        scores_to_fusion = []
        for single_fold_det in all_dets:
            if image_id in single_fold_det.keys():
                box, class_id, score = getNormBoxes(single_fold_det, image_id, width, height)
                boxes_to_fusion.append(box)
                class_to_fusion.append(class_id)
                scores_to_fusion.append(score)
        if len(boxes_to_fusion) >= 4:
            print('fushion')
            # boxes, scores, labels = weighted_boxes_fusion(boxes_to_fusion, scores_to_fusion, class_to_fusion, weights=[1, 2, 3, 2, 2], iou_thr=iou_thr, skip_box_thr=skip_box_thr)
#             boxes, scores, labels = weighted_boxes_fusion(boxes_to_fusion, scores_to_fusion, class_to_fusion, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
            boxes, scores, labels = nms(boxes_to_fusion, scores_to_fusion, class_to_fusion, weights=weights, iou_thr=iou_thr)
        else:
            boxes = boxes_to_fusion[0]
            scores = scores_to_fusion[0]
            labels = class_to_fusion[0]
        json_str = toJson(image_id, boxes, labels, scores, width, height)
        data += json_str
    return data

In [7]:

json_dir = '/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict_test'
output_name = '/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict_test/ens_nms.json'

json_wildcard = os.path.join(json_dir, "*.json")
json_files = glob.glob(json_wildcard)
all_dets, weights = load_all_dets(json_files)
final_preds = wbf(all_dets, iou_thr=0.4, skip_box_thr=0.001, weights = None)
with open(output_name, 'w') as f:
    json.dump(final_preds, f)

/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict_test/fold3.json 1
/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict_test/fold1.json 1
/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict_test/fold2.json 1
/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict_test/fold4.json 1
/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/layer3_predict_test/fold5.json 1
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
Warning. Removed 2 boxes with zero area!
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion
fushion


In [15]:
f = open('/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/runs_test/ens/ens_nms.json', 'r')
data = json.load(f)

In [17]:
data[0]

{'image_id': '4112f6a0a3a7509212801f2b3132dfda.dicom',
 'category_id': 0,
 'score': 0.008172,
 'bbox': [919, 613, 655, 526]}

In [24]:
from ensemble_boxes import *

boxes_list = [[
    [0.00, 0.51, 0.81, 0.91],
    [0.10, 0.31, 0.71, 0.61],
    [0.01, 0.32, 0.83, 0.93],
    [0.02, 0.53, 0.11, 0.94],
    [0.03, 0.24, 0.12, 0.35],
],[
    [0.04, 0.56, 0.84, 0.92],
    [0.12, 0.33, 0.72, 0.64],
    [0.38, 0.66, 0.79, 0.95],
    [0.08, 0.49, 0.21, 0.89],
]]
scores_list = [[0.9, 0.8, 0.2, 0.4, 0.7], [0.5, 0.8, 0.7, 0.3]]
labels_list = [[0, 1, 0, 1, 1], [1, 1, 1, 0]]
weights = None

iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1

boxes, scores, labels = nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)

In [25]:
boxes

array([[0.  , 0.51, 0.81, 0.91],
       [0.08, 0.49, 0.21, 0.89],
       [0.12, 0.33, 0.72, 0.64],
       [0.38, 0.66, 0.79, 0.95],
       [0.03, 0.24, 0.12, 0.35],
       [0.04, 0.56, 0.84, 0.92],
       [0.02, 0.53, 0.11, 0.94]])

In [26]:
scores

array([0.9, 0.3, 0.8, 0.7, 0.7, 0.5, 0.4])